In [1]:
# we can start with the first step.
import pandas as pd
import numpy as np

In [2]:
#import dataset 
df=pd.read_csv("Formula1_Datasets/f1_driver_race_features.csv")
#Filter one driver and race 
driver_name = "Sebastian Vettel"
race_name   = "Australian Grand Prix"
race_year = 2019

clean_sample=(
    df[(df["driver_name"] == driver_name) &
       (df["name"] == race_name) &
       (df["year"] == race_year)]
    .sort_values("lap")
    .reset_index(drop=True))
print(f"Driver–Race subset: {clean_sample.shape[0]} laps")
clean_sample.head(5)

Driver–Race subset: 57 laps


,raceId,year,round,name,date,circuitId,driverId,driver_name,nationality,lap,position,lap_time,milliseconds,lag_1,lag_2,lag_3,rolling_mean_3,rolling_std_3,lap_delta
0,1010,2019,1,Australian Grand Prix,2019-03-17,1,20,Sebastian Vettel,German,2,3,1:30.095,90095,95626.0,NaN,NaN,NaN,NaN,-5531.0
1,1010,2019,1,Australian Grand Prix,2019-03-17,1,20,Sebastian Vettel,German,3,3,1:29.371,89371,90095.0,95626.0,NaN,91697.333333,3421.528947,-724.0
2,1010,2019,1,Australian Grand Prix,2019-03-17,1,20,Sebastian Vettel,German,4,3,1:29.124,89124,89371.0,90095.0,95626.0,89530.000000,504.649383,-247.0
3,1010,2019,1,Australian Grand Prix,2019-03-17,1,20,Sebastian Vettel,German,5,3,1:29.310,89310,89124.0,89371.0,90095.0,89268.333333,128.663644,186.0
4,1010,2019,1,Australian Grand Prix,2019-03-17,1,20,Sebastian Vettel,German,6,3,1:29.666,89666,89310.0,89124.0,89371.0,89366.666667,275.407577,356.0


In [3]:
#let's scale the target 
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
clean_sample["scaled_time"] = scaler.fit_transform(clean_sample[["milliseconds"]])

clean_sample[["lap", "milliseconds", "scaled_time"]].head()

,lap,milliseconds,scaled_time
0,2,90095,0.114658
1,3,89371,0.075885
2,4,89124,0.062657
3,5,89310,0.072618
4,6,89666,0.091683


In [4]:
def create_sequence(series,time_step):
    x,y=[],[]
    for i in range(len(series)-time_step):
        x.append(series[i:i+time_step])
        y.append(series[i + time_step])
    return(np.array(x),np.array(y))


In [5]:
# Convert to time-series arrays
series = clean_sample["scaled_time"].values
# Create sliding windows (past 3 laps → predict next)
x, y = create_sequence(series, time_step=5)
# Reshape for LSTM [samples, time_steps, features]
x = x.reshape(x.shape[0], x.shape[1], 1)
print("Sequence shapes created successfully!")
print("X shape:", x.shape, "| y shape:", y.shape)

Sequence shapes created successfully!
X shape: (52, 5, 1) | y shape: (52,)


In [6]:
#now we can train the model
train_size=int(len(x)*0.8)
x_train,x_test=x[:train_size],x[train_size:]
y_train,y_test = y[:train_size], y[train_size:]
print(f"Train samples: {x_train.shape[0]}, Test samples: {x_test.shape[0]}")

Train samples: 41, Test samples: 11


In [7]:
# we can build an LSTM model.
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Dense,Dropout
from tensorflow.keras.callbacks import EarlyStopping

time_step = 5 
model = Sequential([
    LSTM(64, activation="tanh", input_shape=(time_step, 1)),
    Dropout(0.15),
    Dense(32, activation="relu"),
    Dense(1)     # output layer
])
model.compile(optimizer="adam", loss="mse", metrics=["mae"])
model.summary()



C:\Users\prana\anaconda3\envs\fresh_env\lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                          │ (None, 64)                  │          16,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 32)                  │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │              33 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 19,009 (74.25 KB)

 Trainable params: 19,009 (74.25 KB)

 Non-trainable params: 0 (0.00 B)

In [8]:
es = EarlyStopping(monitor="val_loss", patience=8,
                   restore_best_weights=True)
history = model.fit(
    x_train, y_train,
    validation_split=0.1,
    epochs=60,
    batch_size=8,
    callbacks=[es],
    verbose=2
)


Epoch 1/60
5/5 - 7s - 1s/step - loss: 0.0321 - mae: 0.0711 - val_loss: 1.0663e-04 - val_mae: 0.0096
Epoch 2/60
5/5 - 0s - 95ms/step - loss: 0.0281 - mae: 0.0531 - val_loss: 0.0011 - val_mae: 0.0316
Epoch 3/60
5/5 - 0s - 71ms/step - loss: 0.0272 - mae: 0.0619 - val_loss: 0.0030 - val_mae: 0.0548
Epoch 4/60
5/5 - 0s - 49ms/step - loss: 0.0271 - mae: 0.0762 - val_loss: 0.0042 - val_mae: 0.0642
Epoch 5/60
5/5 - 0s - 52ms/step - loss: 0.0275 - mae: 0.0799 - val_loss: 0.0035 - val_mae: 0.0586
Epoch 6/60
5/5 - 0s - 53ms/step - loss: 0.0270 - mae: 0.0708 - val_loss: 0.0023 - val_mae: 0.0471
Epoch 7/60
5/5 - 0s - 91ms/step - loss: 0.0268 - mae: 0.0636 - val_loss: 0.0020 - val_mae: 0.0441
Epoch 8/60
5/5 - 1s - 113ms/step - loss: 0.0273 - mae: 0.0615 - val_loss: 0.0015 - val_mae: 0.0384
Epoch 9/60
5/5 - 0s - 35ms/step - loss: 0.0264 - mae: 0.0589 - val_loss: 0.0022 - val_mae: 0.0462


In [9]:
#forecast and inverse transform
y_pred_scaled = model.predict(x_test)
y_pred = scaler.inverse_transform(y_pred_scaled)
y_test_inv = scaler.inverse_transform(y_test.reshape(-1, 1))


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 354ms/step


In [10]:
#evlaute performance.
from sklearn.metrics import mean_absolute_error, mean_squared_error
mae = mean_absolute_error(y_test_inv, y_pred)
rmse = np.sqrt(mean_squared_error(y_test_inv, y_pred))
print(f"\n📊 LSTM Results for {driver_name} - {race_name}")
print(f"MAE : {mae:.2f} ms")
print(f"RMSE: {rmse:.2f} ms")



📊 LSTM Results for Sebastian Vettel - Australian Grand Prix
MAE : 721.90 ms
RMSE: 992.77 ms


In [34]:
import joblib
model.save("f1_lstm_model.h5")
print("model saved as f1_lstm_model.h5")


model saved as f1_lstm_model.h5


In [36]:
joblib.dump(scaler,"scaler_target.pkl")

['scaler_target.pkl']

In [38]:
#inorder to create a minimal Deployment ready dataset.
#Create a small subset for Streamlit demo
subset = clean_sample[['driver_name','name','year','lap','milliseconds','lag_1','rolling_mean_3','lap_delta']].copy()
# save it to CSV
subset.to_csv("sample_data.csv", index=False)
print("Rows saved:", subset.shape[0])
print("✅ sample_data.csv created successfully.")

Rows saved: 57
✅ sample_data.csv created successfully.
